In [10]:
import os
import requests
import pandas as pd

In [11]:
def get_github_profile_details(username):
    # Endpoint to get user details
    user_url = f"https://api.github.com/users/{username}"
    response = requests.get(user_url)
    user_details = response.json()

    # Endpoint to get user repositories
    repos_url = f"https://api.github.com/users/{username}/repos"
    response = requests.get(repos_url)
    repos_details = response.json()

    # Collecting repository details including topics
    repositories = []
    for repo in repos_details:
        repo_topics_url = f"https://api.github.com/repos/{username}/{repo['name']}/topics"
        headers = {'Accept': 'application/vnd.github.mercy-preview+json'}  
        topics_response = requests.get(repo_topics_url, headers=headers)
        topics_details = topics_response.json().get('names', [])  

        repositories.append({
            "name": repo["name"],
            "url": repo["html_url"],
            "description": repo["description"],
            "stars": repo["stargazers_count"],
            "topics": topics_details
        })

    return {
        "user_details": user_details,
        "repositories": repositories
    }


In [12]:
profile_info = get_github_profile_details("pgplarosa")

In [13]:
repo_details = pd.DataFrame(profile_info["repositories"])

In [15]:
# configs
root_dir = os.path.expanduser("~")
github_path = "curriebot/raw/github/"
github_full_path = os.path.join(root_dir, github_path)

if not os.path.isdir(github_full_path):
    os.makedirs(github_full_path)
    
repo_details.to_csv(os.path.join(github_full_path, "github_summary.csv"), index=False)